<a href="https://colab.research.google.com/github/PyThaiNLP/thaimaimeex/blob/master/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ThaiME Project Name Generation

This notebook allows you to generate ThaiME project names based on the language model trained on over 30,000 ThaiME project names. You can also try naming your own project and have it estimated by a LinearSVC model trained on the [thaimaimeex](https://github.com/PyThaiNLP/thaimaimeex) dataset.

## Setups

Run all cells to prepare.

In [1]:
#@title
#download and installs for colab
!npx degit pythainlp/thaimaimeex -f
!pip install emoji
!pip install pythainlp -q

npx: installed 1 in 3.414s
> destination directory is not empty. Using --force, continuing
> cloned pythainlp/thaimaimeex#master
     |████████████████████████████████| 51kB 2.0MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=5d1bda41d568f30aa73ea121dd1d8994e1f21c9d1efee0c8c4c946a6e400d9e5
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji
     |████████████████████████████████| 11.1MB 235kB/s 
     |████████████████████████████████| 1.4MB 37.3MB/s 


In [0]:
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm import tqdm_notebook
from collections import Counter
from joblib import dump, load
import re

#viz
import matplotlib.pyplot as plt
import seaborn as sns

#fastai
import fastai
from fastai.text import *
from fastai.callbacks import CSVLogger

#pythainlp
from pythainlp.ulmfit import *

In [0]:
# #budget estimator
# !wget https://github.com/PyThaiNLP/pythainlp-corpus/releases/download/thaime_lm_lstm-v.01/thaime_clf.joblib
# #lm data
# !wget https://github.com/PyThaiNLP/pythainlp-corpus/releases/download/thaime_lm_lstm-v.01/thaime_lm.pkl
# #lm model

In [26]:
clf = load('thaime_clf.joblib') 
np.exp(clf.predict(['โครงการพัฒนาผลิตภัณฑ์ชุมชน']))

array([2124051.585378])

In [0]:
data_lm = load_data('','thaime_lm.pkl')
data_lm.sanity_check()

#check vocab size
len(data_lm.vocab.itos)

config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False, tie_weights=True, out_bias=True,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(drop_mult=0.9, clip=0.12, alpha=2, beta=1)

learn = language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, **trn_args)

#load pretrained models
learn.load('thaime_lm')

## Generate Project Names

Generate project names with some parameters.

In [59]:
#@title {run: "auto"}
Length = 20 #@param {type:"slider", min:10, max:200, step:1}
Temperature = 0.8 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
Seed_Word = 'โครงการ' #@param {type:"string"}
nb_ = 20

for i in range(nb_):
    t = learn.predict(Seed_Word, 
                      Length, 
                      temperature=Temperature, 
                      min_p=0.005, sep = '')
    est = np.round(np.exp(clf.predict([t])[0])/1e6,2)
    print(f'{t} ราคา:{est}M')

โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก สายบ้านกุดสิมโคก หมู่ที่ 6 (สายทางถนน ราคา:0.57M
โครงการเสริมผิวลาดยางแอสฟัลท์ติกคอนกรีต (สายบ้านนายบุญคง - ราคา:1.11M
โครงการก่อสร้างถนนคอนกรีตเสริมเหล็กสายบ้านนายบุญเกิด หมู่ที่ 6 - หมู่ที่ 3 ราคา:1.03M
โครงการ ติดตั้งระบบ โซล่าเซลล์ เพื่อพัฒนาโครงสร้างพื้นฐานและโครงสร้างพื้นฐาน ทางหลวงชนบท หมายเลข 004 ราคา:12.64M
โครงการซ่อมสร้างผิวทางแอสฟัลท์ติกคอนกรีต สาย ยส.ถ.1- ถ. ราคา:4.71M
โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก สายซอยถนน 2 หมู่ที่ 4 บ้านหัวหิน ตำบล ราคา:1.35M
โครงการก่อสร้างถนนแอสฟัลท์ติกคอนกรีต ซอย 8 หมู่ที่ 10 บ้าน ราคา:1.99M
โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก สายหนอง นา ม.11 บ้านท่าหิน หมู่ที่ 1 ราคา:0.85M
โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก สายบ้านกุดเชียงหมี หมู่ที่ 9 - บ้านทุ่งกลาง ราคา:2.02M
โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก หมู่ที่ 3 บ้านบางอ้อ ต.บางใหญ่ อ.บางสะพาน ราคา:2.52M
โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก สายบ้านคลอง - ห้วยหนองบอน หมู่ที่ 8 บ้าน ราคา:1.14M
โครงการเสริมผิวจราจรแอสฟัลท์คอนกรีต สาย ยส.ถ.1- บ้านหนองบัว - บ้าน ราคา:

## DIY Project Names

Enter your own project names and see how much budget it would have.

In [62]:
Project_Name = 'โครงการอะไรสักอย่าง' #@param {type:"string"}
cost = round(np.exp(clf.predict([Project_Name])[0])/1e6,2)
print(f'{cost}M THB')

4.38M THB
